> Tah Plen

for each feature and each label...
- model (simplify) the data to known distributions (gaussian, etc)
- calculate probability that input data (single feature) fits to given distribution

analyzing probability through multiple feature and label are expected to give prediction in method such as... (what should be done after this)
- for each feature, calculate each probability where it belongs to 0 or 1 and multiply them for each class
- assume the class only based on if probability exceed certain threshold for certain class
- and more

so for this code,

when making a model...

input requires two lists as below

    [list of values in single feature with class 0], [" with class 1] (, might have more inputs to define model like defining (assuming) distribution)

and output will be a 
    
    list/dictionary that summerizes the distribution

(it will feed directly to below without going anywhere, so i'll take this on my own)

when making a prediction...

input includes
    
    data to predict, a summarized model

output will be a list with two elements

    [probability of data included in class 0, " class 1]

note about bayes

P( has specific lable | has specific value ) = P( has specific value | has specific label ) * P( has specific label ) / P( has specific value )

- interest: P( label | value )


- P( value | label ): from modeling the data distribution with specific label
- P( label ): from ratio
- P( value ): from modeling the whole data distribution

In [27]:
import math

def mean(numbers: list) -> float:
    """Bayes: Calculate the mean of a list of numbers
    
    Requires math library
    
    from machinelearningmastery"""
    
    return sum(numbers)/float(len(numbers))

def stdev(numbers: list) -> float:
    """Bayes: Calculate the standard deviation of a list of numbers
    
    Requires math library
    
    from machinelearningmastery"""
    avg = mean(numbers)
    variance = sum([(x-avg)**2 for x in numbers]) / float(len(numbers)-1)
    return math.sqrt(variance)

def summarize_gaussian(numbers: list) -> dict:
    """Bayes: Calculate the mean, stdev, and len for numbers which is a summerized gaussian model
    
    Requires math library and means, stdev function
    
    modified from machinelearningmastery"""
    summaries = {'model': 'gaussian', 'mean': mean(numbers), 'stdev': stdev(numbers), 'len': len(numbers)}
    return summaries

In [29]:
def calculate_probability_gaussian(x: float, mean: float, stdev: float) -> float:
    """Bayes: Calculate the Gaussian probability distribution function for x
    
    Requires math library
    
    from machinelearningmastery"""
    x = float(x)
    mean = float(mean)
    stdev = float(mean)
    exponent = math.exp(-((x - mean)**2 / (2 * stdev**2 )))
    return (1 / (math.sqrt(2 * math.pi) * stdev)) * exponent
 
def calculate_class_probabilities(target: float, total_summary: dict, label0_summary: dict, label1_summary: dict) -> tuple:
    """Bayes: Calculate the probabilities of target belong in label0 and label1
    
    Requires calculate_probability_gaussian function"""
    # if model != gaussian -> return -1
    px0 = calculate_probability_gaussian(target, label0_summary['mean'], label0_summary['stdev'])
    px1 = calculate_probability_gaussian(target, label1_summary['mean'], label1_summary['stdev'])
    p0 = label0_summary['len'] / total_summary['len']
    p1 = label1_summary['len'] / total_summary['len']
    px = calculate_probability_gaussian(target, total_summary['mean'], total_summary['stdev'])
    p0x = px0 * p0 #/ px
    p1x = px1 * p1 #/ px
    return p0x, p1x

In [30]:
def thatfunctionthatconsiderspossibilityofmultiplefeatureandchoosetheclasswhichgivendatabelongstoandalsohascomicallylongname(probabilities: list, debug: int = 0) -> int: # bias: list = [],
    """Bayes: Calculates combinations of possibilities that certain value belongs to each features while not others
    
    Add each probability for when each class is dominant and compare the value in order to predict the label
    
    probabilities
        list of tuples from calculate_class_probabilities
    
    debug
        when value is 1, print the values of each possibilities and other detailed statistics
    """
    
    # 예시: 5개의 feature 중 예측된 레이블이 각 feature 순서대로 첫 2개에서 0, 뒷 3개에서 1이라고 예측했을 때 실제 값이 0일 확률은
    # (1번 레이블에서 0이라고 예측한 게 맞을 확률) * (2번 ") * (3번 레이블에서 1이라고 예측한 게 틀릴 확률) * ...
    # 틀릴 확률 = 1 - 맞을 확률
    
    # bias (X)
    #     Use to multiply the consideration to certain feature
        
    #     when [1, 1, 2, 1, 1], third feature will be considered twice than other features
    
    iszero = 0
    isone = 0
    featurecount = len(probabilities)
    
    # if len(bias) != featurecount:
    #     bias = [1] * featurecount
    
    # if debug == 1:
    #     for b in range(len(bias)):
    #         if b != 1:
    #             print(f"bias in feature #{b}: x{bias[b]}")
    
    for i in range(0, 2**featurecount):
        currentcombination = eval("'{:0>" + str(featurecount) + "}'.format(bin(" + str(i) + ")[2:])")
        iscurrentcombinationofzero = 1
        iscurrentcombinationofone = 1
        for j in range(featurecount):
            if currentcombination[j] == '0':
                iscurrentcombinationofzero *= probabilities[j][0] # * bias[j]
                iscurrentcombinationofone *= (1 - probabilities[j][0])
            else:
                iscurrentcombinationofzero *= (1 - probabilities[j][1]) # * bias[j]
                iscurrentcombinationofone *= probabilities[j][1]
        iszero += iscurrentcombinationofzero
        isone += iscurrentcombinationofone
        if debug:
            print(f"combination: {currentcombination} | probability of label being 0 {iscurrentcombinationofzero}")
            print(" " * len(f"combination: {currentcombination} ") + f"| probability of label being 0 {iscurrentcombinationofone}")
    if debug:
        print(f"total probability of being 0: {iszero}")
        print(f"total probability of being 1: {isone}")
    if iszero >= isone:
        return 0
    else:
        return 1

In [24]:
def thatfunctionthatconsiderspossibilityofmultiplefeatureandchoosetheclasswhichgivendatabelongstoandalsohascomicallylongname(probabilities: list, debug: int = 0) -> int:
    iszero = 0
    isone = 0
    for probability in probabilities:
        if probability[0] >= probability[1]:
            iszero += 1
        else:
            isone += 1
    
    if probabilities[-1][0] >= probabilities[-1][1]:
        iszero += len(probabilities) -2
    else:
        isone += len(probabilities) -2
    
    if iszero < isone:
        return 1
    else:
        return 0

In [7]:
def summerize_multiple_features(featured: list) -> list:
    """Bayes: summerize each feature in featured with gaussian and return list of summerized models
    
    Requires summerize gaussian function
    
    elements of returned list is a list with three elements, each are summary of total data, data with label 0, and data with label 1"""
    model_summeries = []

    for feature_no in range(len(featured)):
        labelzero = summarize_gaussian(featured[feature_no][0])
        labelone = summarize_gaussian(featured[feature_no][1])
        total = featured[feature_no][0] + featured[feature_no][1]
        labeltotal = summarize_gaussian(total)
        model_summeries.append([labeltotal, labelzero, labelone])
    
    return model_summeries

In [8]:
#example of input when training model
# featured_example = [[['values of feature #1 when label = 0'], ['values of feature #1 when label = 1']], [['values of feature #2 when label = 0'], ['values of feature #2 when label = 1']], ['...']]

In [ ]:
# modelmaking

datas_example = [] #featured_example-like data

summeries_example = summerize_multiple_features(datas_example)

In [31]:
def predict(data: list, summary: list) -> int:
    """Bayes: predict the label of single sets of data when summary"""
    
    if len(data) != len(summary):
        print("Bayes: number of feature does not match!")
        return -1

    perchance = []

    for featurenum in range(len(summary)):
        perchance.append(calculate_class_probabilities(data[featurenum], summary[featurenum][0], summary[featurenum][1], summary[featurenum][2]))
    
    print(perchance)

    return thatfunctionthatconsiderspossibilityofmultiplefeatureandchoosetheclasswhichgivendatabelongstoandalsohascomicallylongname(perchance)

In [10]:
def predict_multiple(datas: list, summary: list) -> list:
    """Bayes: predict multiple datas and return the values as list"""
    
    predictions = []
    
    for data in datas:
        predictions.append(predict(data, summary))
        
    return predictions

In [11]:
def calculate_f1(original_label: list, predicted_label: list) -> float:
    if len(original_label) != len(predicted_label):
        return -1
    
    # positive = 1 / negative = 0
    
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    
    for i in range(len(original_label)):
        if original_label[i] == predicted_label[i]:
            if predicted_label[i] == 1:
                TP += 1
            else:
                TN += 1
        else:
            if predicted_label[i] == 1:
                FP += 1
            else:
                FN += 1
    
    if (TP + FP):
        pre = TP / (TP + FP)
    else:
        pre = 0
    if (TP + FN):
        rec = TP / (TP + FN)
    else:
        rec = 0
    if (TP + FN + FP + TN):
        acc = (TP + TN) / (TP + FN + FP + TN)
    else:
        acc = 0
    if (pre + rec):
        f1 = 2 * pre * rec / (pre + rec)
    else:
        f1 = 0
    
    print(f"precision: {pre}")
    print(f"recall: {rec}")
    print(f"accuracy: {acc}")
    print(f"f1-score: {f1}")
    
    return pre, rec, acc, f1

In [12]:
# label_0_AQI,label_1_AQI,label_0_sun_set,label_1_sun_set,label_0_fine,label_1_fine,label_0_night_duration,label_1_night_duration,label_0_day_duration,label_1_day_duration
feature_set_1_data_2021 = [
    [
        [
            93,
            122,
            127,
            129,
            95,
            95,
            127,
            89,
            97,
            119,
            137,
            155,
            180,
            174,
            161,
            152,
            87,
            95,
            99,
            151,
            124,
            87,
            38,
            46,
            72,
            97,
            107,
            102,
            91,
            87,
            76,
            74,
            151,
            158,
            80,
            70,
            89,
            147,
            158,
            162,
            166,
            134,
            70,
            110,
            112,
            107,
            152,
            156,
            137,
            137,
            122,
            129,
            89,
            59,
            70,
            53,
            38,
            91,
            99,
            99,
            74,
            68,
            102,
            115,
            161,
            174,
            153,
            132,
            156,
            157,
            156,
            167,
            167,
            155,
            78,
            112,
            105,
            158,
            144,
            151,
            156,
            142,
            61,
            332,
            192,
            102,
            112,
            124,
            82,
            38,
            68,
            102,
            124,
            91,
            89,
            91,
            112,
            91,
            53,
            78,
            99,
            84,
            162,
            89,
            119,
            152,
            160,
            158,
            112,
            72,
            74,
            82,
            82,
            152,
            154,
            95,
            72,
            57,
            87,
            74,
            122,
            156,
            399,
            468,
            137,
            68,
            63,
            70,
            115,
            134,
            112,
            21,
            33,
            78,
            87,
            72,
            50,
            76,
            87,
            186,
            172,
            117,
            107,
            95,
            68,
            74,
            93,
            102,
            89,
            80,
            119,
            152,
            97,
            110,
            112,
            137,
            70,
            72,
            107,
            74,
            38,
            59,
            57,
            110,
            129,
            105,
            72,
            63,
            91,
            102,
            68,
            78,
            89,
            97,
            78,
            46,
            29,
            68,
            80,
            61,
            78,
            87,
            63,
            50,
            68,
            80,
            57,
            53,
            55,
            61,
            53,
            76,
            78,
            46,
            59,
            107,
            91,
            95,
            84,
            102,
            84,
            25,
            50,
            53,
            38,
            42,
            55,
            38,
            42,
            87,
            68,
            76,
            63,
            55,
            42,
            38,
            38,
            46,
            46,
            57,
            46,
            57,
            57,
            29,
            25,
            38,
            42,
            53,
            66,
            97,
            78,
            97,
            105,
            105,
            25,
            55,
            66,
            42,
            68,
            80,
            42,
            46,
            66,
            78,
            91,
            78,
            91,
            95,
            115,
            105,
            127,
            153,
            152,
            95,
            38,
            38,
            57,
            66,
            70,
            82,
            134,
            152,
            139,
            102,
            142,
            169,
            188,
            176,
            115,
            93,
            82,
            129,
            80,
            102,
            124,
            137,
            72,
            63,
            74,
            72,
            74,
            95,
            107,
            134,
            112,
            97,
            84,
            155,
            115,
            72,
            93,
            161,
            164,
            99,
            76,
            97,
            144,
            151,
            95,
            127,
            132,
            78,
            66,
            91,
            129,
            151,
            97,
            78,
        ],  # label_0_AQI
        [
            127,
            91,
            66,
            99,
            112,
            72,
            87,
            93,
            72,
            66,
            55,
            55,
            68,
            68,
            74,
            82,
            78,
            68,
            82,
            95,
            124,
            82,
            74,
            68,
            74,
            76,
            66,
            57,
            63,
            50,
            61,
            97,
            102,
            80,
            42,
            63,
            57,
            46,
            76,
            87,
        ],
    ],  # label_1_AQI
    [
        [
            1044,
            1045,
            1045,
            1046,
            1047,
            1048,
            1049,
            1050,
            1051,
            1052,
            1053,
            1054,
            1055,
            1056,
            1057,
            1058,
            1059,
            1060,
            1061,
            1062,
            1063,
            1064,
            1065,
            1066,
            1067,
            1068,
            1069,
            1071,
            1076,
            1077,
            1078,
            1079,
            1081,
            1082,
            1083,
            1084,
            1085,
            1086,
            1087,
            1088,
            1089,
            1090,
            1091,
            1092,
            1094,
            1095,
            1096,
            1097,
            1098,
            1099,
            1100,
            1101,
            1102,
            1103,
            1104,
            1106,
            1107,
            1108,
            1109,
            1110,
            1111,
            1112,
            1113,
            1114,
            1115,
            1116,
            1116,
            1117,
            1118,
            1119,
            1120,
            1121,
            1122,
            1123,
            1124,
            1125,
            1126,
            1127,
            1128,
            1129,
            1129,
            1130,
            1131,
            1132,
            1133,
            1134,
            1135,
            1136,
            1137,
            1138,
            1138,
            1139,
            1140,
            1141,
            1142,
            1143,
            1144,
            1145,
            1146,
            1147,
            1148,
            1148,
            1149,
            1150,
            1151,
            1152,
            1153,
            1154,
            1155,
            1156,
            1157,
            1158,
            1158,
            1159,
            1160,
            1161,
            1162,
            1163,
            1164,
            1165,
            1166,
            1167,
            1168,
            1168,
            1169,
            1170,
            1171,
            1172,
            1173,
            1174,
            1175,
            1175,
            1176,
            1177,
            1178,
            1179,
            1180,
            1180,
            1181,
            1182,
            1183,
            1184,
            1184,
            1185,
            1186,
            1186,
            1187,
            1188,
            1189,
            1190,
            1190,
            1191,
            1191,
            1192,
            1192,
            1193,
            1193,
            1194,
            1195,
            1195,
            1195,
            1196,
            1196,
            1196,
            1197,
            1197,
            1197,
            1197,
            1197,
            1197,
            1197,
            1197,
            1197,
            1197,
            1197,
            1197,
            1197,
            1197,
            1196,
            1196,
            1195,
            1195,
            1193,
            1191,
            1186,
            1180,
            1179,
            1166,
            1159,
            1158,
            1157,
            1155,
            1154,
            1153,
            1151,
            1150,
            1148,
            1147,
            1146,
            1144,
            1143,
            1141,
            1140,
            1138,
            1137,
            1135,
            1134,
            1132,
            1131,
            1128,
            1125,
            1123,
            1121,
            1120,
            1118,
            1117,
            1115,
            1114,
            1112,
            1111,
            1109,
            1108,
            1106,
            1104,
            1103,
            1101,
            1100,
            1098,
            1097,
            1095,
            1094,
            1092,
            1091,
            1089,
            1088,
            1086,
            1085,
            1083,
            1082,
            1080,
            1079,
            1078,
            1076,
            1075,
            1057,
            1056,
            1055,
            1054,
            1053,
            1052,
            1051,
            1050,
            1049,
            1048,
            1047,
            1046,
            1045,
            1044,
            1043,
            1043,
            1042,
            1041,
            1040,
            1040,
            1039,
            1039,
            1038,
            1037,
            1037,
            1037,
            1036,
            1036,
            1035,
            1035,
            1035,
            1034,
            1034,
            1034,
            1034,
            1034,
            1034,
            1033,
            1033,
            1033,
            1034,
            1034,
            1034,
            1034,
            1034,
            1034,
            1035,
            1035,
            1035,
            1035,
            1036,
            1036,
            1037,
            1037,
            1038,
            1038,
            1039,
            1039,
            1040,
            1041,
            1041,
            1042,
            1043,
            1044,
        ],  # label_0_sun_set
        [
            1188,
            1194,
            1197,
            1197,
            1196,
            1196,
            1194,
            1194,
            1194,
            1193,
            1192,
            1191,
            1190,
            1189,
            1189,
            1188,
            1187,
            1186,
            1185,
            1184,
            1183,
            1182,
            1181,
            1178,
            1177,
            1176,
            1175,
            1174,
            1173,
            1172,
            1171,
            1169,
            1168,
            1167,
            1165,
            1163,
            1162,
            1161,
            1129,
            1126,
        ],
    ],  # label_1_sun_set
    [
        [
            13.0,
            9.0,
            11.0,
            19.0,
            12.0,
            11.0,
            9.0,
            8.0,
            13.0,
            19.0,
            27.0,
            40.0,
            36.0,
            27.0,
            33.0,
            34.0,
            13.0,
            15.0,
            13.0,
            22.0,
            26.0,
            22.0,
            6.0,
            8.0,
            14.0,
            23.0,
            18.0,
            16.0,
            18.0,
            18.0,
            15.0,
            14.0,
            36.0,
            45.0,
            17.0,
            13.0,
            21.0,
            41.0,
            52.0,
            57.0,
            66.0,
            35.0,
            13.0,
            12.0,
            14.0,
            20.0,
            32.0,
            40.0,
            25.0,
            15.0,
            19.0,
            28.0,
            19.0,
            9.0,
            13.0,
            9.0,
            6.0,
            18.0,
            21.0,
            22.0,
            15.0,
            14.0,
            24.0,
            29.0,
            58.0,
            82.0,
            43.0,
            35.0,
            55.0,
            57.0,
            16.0,
            18.0,
            37.0,
            24.0,
            12.0,
            31.0,
            16.0,
            24.0,
            22.0,
            27.0,
            40.0,
            30.0,
            13.0,
            41.0,
            26.0,
            16.0,
            12.0,
            12.0,
            11.0,
            5.0,
            10.0,
            21.0,
            29.0,
            16.0,
            16.0,
            19.0,
            24.0,
            19.0,
            8.0,
            13.0,
            20.0,
            13.0,
            18.0,
            13.0,
            20.0,
            30.0,
            35.0,
            36.0,
            20.0,
            11.0,
            12.0,
            12.0,
            13.0,
            22.0,
            20.0,
            16.0,
            12.0,
            7.0,
            17.0,
            14.0,
            11.0,
            17.0,
            58.0,
            66.0,
            12.0,
            9.0,
            8.0,
            10.0,
            23.0,
            30.0,
            30.0,
            3.0,
            3.0,
            13.0,
            18.0,
            12.0,
            6.0,
            14.0,
            19.0,
            24.0,
            21.0,
            13.0,
            16.0,
            17.0,
            10.0,
            12.0,
            16.0,
            21.0,
            17.0,
            11.0,
            23.0,
            34.0,
            17.0,
            24.0,
            25.0,
            34.0,
            13.0,
            12.0,
            25.0,
            15.0,
            4.0,
            9.0,
            10.0,
            26.0,
            32.0,
            25.0,
            12.0,
            12.0,
            20.0,
            25.0,
            14.0,
            15.0,
            22.0,
            24.0,
            17.0,
            6.0,
            4.0,
            12.0,
            18.0,
            10.0,
            18.0,
            21.0,
            11.0,
            7.0,
            14.0,
            21.0,
            8.0,
            9.0,
            9.0,
            12.0,
            8.0,
            17.0,
            17.0,
            6.0,
            8.0,
            23.0,
            17.0,
            19.0,
            19.0,
            25.0,
            19.0,
            1.0,
            3.0,
            3.0,
            2.0,
            2.0,
            3.0,
            2.0,
            2.0,
            18.0,
            14.0,
            15.0,
            10.0,
            7.0,
            5.0,
            5.0,
            5.0,
            6.0,
            7.0,
            9.0,
            6.0,
            8.0,
            9.0,
            3.0,
            3.0,
            4.0,
            6.0,
            7.0,
            11.0,
            20.0,
            14.0,
            24.0,
            20.0,
            16.0,
            3.0,
            8.0,
            11.0,
            5.0,
            12.0,
            10.0,
            4.0,
            5.0,
            10.0,
            13.0,
            18.0,
            14.0,
            20.0,
            23.0,
            29.0,
            25.0,
            34.0,
            41.0,
            37.0,
            21.0,
            4.0,
            6.0,
            8.0,
            8.0,
            10.0,
            14.0,
            26.0,
            26.0,
            24.0,
            20.0,
            26.0,
            56.0,
            89.0,
            76.0,
            15.0,
            11.0,
            13.0,
            18.0,
            13.0,
            21.0,
            31.0,
            33.0,
            16.0,
            11.0,
            14.0,
            10.0,
            13.0,
            23.0,
            24.0,
            32.0,
            24.0,
            21.0,
            18.0,
            43.0,
            25.0,
            11.0,
            19.0,
            50.0,
            52.0,
            16.0,
            13.0,
            15.0,
            27.0,
            31.0,
            18.0,
            27.0,
            31.0,
            13.0,
            10.0,
            17.0,
            32.0,
            38.0,
            18.0,
            8.0,
        ],  # label_0_fine
        [
            27.0,
            20.0,
            12.0,
            24.0,
            26.0,
            15.0,
            20.0,
            21.0,
            14.0,
            12.0,
            9.0,
            8.0,
            11.0,
            13.0,
            16.0,
            20.0,
            19.0,
            16.0,
            20.0,
            27.0,
            36.0,
            20.0,
            16.0,
            11.0,
            14.0,
            16.0,
            14.0,
            9.0,
            12.0,
            6.0,
            10.0,
            26.0,
            28.0,
            19.0,
            7.0,
            13.0,
            10.0,
            6.0,
            3.0,
            21.0,
        ],
    ],  # label_1_fine
    [
        [
            863,
            862,
            862,
            861,
            860,
            859,
            858,
            857,
            856,
            855,
            854,
            853,
            852,
            850,
            849,
            848,
            846,
            845,
            844,
            842,
            841,
            839,
            838,
            836,
            835,
            833,
            831,
            829,
            820,
            818,
            816,
            814,
            811,
            809,
            807,
            805,
            803,
            801,
            799,
            797,
            795,
            793,
            790,
            788,
            785,
            783,
            781,
            778,
            776,
            774,
            772,
            769,
            767,
            765,
            762,
            757,
            755,
            753,
            750,
            748,
            745,
            743,
            741,
            738,
            736,
            733,
            732,
            729,
            727,
            724,
            722,
            719,
            717,
            714,
            712,
            709,
            707,
            704,
            702,
            699,
            698,
            695,
            693,
            690,
            688,
            685,
            683,
            680,
            678,
            675,
            674,
            671,
            669,
            666,
            664,
            661,
            659,
            656,
            654,
            652,
            649,
            648,
            646,
            643,
            641,
            638,
            636,
            634,
            632,
            629,
            627,
            625,
            624,
            621,
            619,
            617,
            615,
            613,
            611,
            608,
            606,
            604,
            602,
            601,
            599,
            597,
            595,
            593,
            591,
            590,
            588,
            587,
            585,
            583,
            582,
            580,
            578,
            578,
            576,
            574,
            573,
            571,
            571,
            569,
            568,
            567,
            566,
            564,
            563,
            561,
            561,
            560,
            560,
            559,
            558,
            557,
            557,
            556,
            555,
            555,
            555,
            554,
            555,
            555,
            554,
            554,
            554,
            555,
            555,
            555,
            555,
            556,
            557,
            557,
            558,
            558,
            559,
            559,
            562,
            563,
            564,
            565,
            570,
            574,
            585,
            596,
            598,
            621,
            632,
            634,
            636,
            639,
            641,
            643,
            645,
            647,
            650,
            652,
            654,
            657,
            658,
            661,
            663,
            666,
            668,
            671,
            672,
            675,
            677,
            682,
            686,
            689,
            692,
            694,
            697,
            699,
            701,
            703,
            706,
            708,
            711,
            713,
            716,
            718,
            720,
            723,
            725,
            728,
            730,
            733,
            734,
            737,
            739,
            742,
            744,
            747,
            749,
            752,
            754,
            756,
            758,
            760,
            763,
            765,
            797,
            799,
            801,
            803,
            805,
            807,
            809,
            811,
            813,
            815,
            817,
            819,
            821,
            824,
            826,
            827,
            829,
            831,
            833,
            834,
            836,
            837,
            839,
            841,
            842,
            843,
            845,
            846,
            848,
            849,
            850,
            852,
            853,
            854,
            855,
            856,
            857,
            859,
            860,
            860,
            860,
            861,
            862,
            863,
            863,
            864,
            864,
            865,
            865,
            866,
            865,
            866,
            866,
            866,
            866,
            866,
            866,
            866,
            865,
            865,
            865,
            864,
            864,
            863,
        ],  # label_0_night_duration
        [
            564,
            556,
            556,
            557,
            561,
            562,
            567,
            567,
            568,
            570,
            572,
            575,
            576,
            578,
            579,
            581,
            582,
            584,
            587,
            589,
            591,
            592,
            594,
            600,
            602,
            603,
            605,
            607,
            609,
            611,
            613,
            615,
            617,
            619,
            623,
            626,
            628,
            629,
            680,
            685,
        ],
    ],  # label_1_night_duration
    [
        [
            577,
            578,
            578,
            579,
            580,
            581,
            582,
            583,
            584,
            585,
            586,
            587,
            588,
            590,
            591,
            592,
            594,
            595,
            596,
            598,
            599,
            601,
            602,
            604,
            605,
            607,
            609,
            611,
            620,
            622,
            624,
            626,
            629,
            631,
            633,
            635,
            637,
            639,
            641,
            643,
            645,
            647,
            650,
            652,
            655,
            657,
            659,
            662,
            664,
            666,
            668,
            671,
            673,
            675,
            678,
            683,
            685,
            687,
            690,
            692,
            695,
            697,
            699,
            702,
            704,
            707,
            708,
            711,
            713,
            716,
            718,
            721,
            723,
            726,
            728,
            731,
            733,
            736,
            738,
            741,
            742,
            745,
            747,
            750,
            752,
            755,
            757,
            760,
            762,
            765,
            766,
            769,
            771,
            774,
            776,
            779,
            781,
            784,
            786,
            788,
            791,
            792,
            794,
            797,
            799,
            802,
            804,
            806,
            808,
            811,
            813,
            815,
            816,
            819,
            821,
            823,
            825,
            827,
            829,
            832,
            834,
            836,
            838,
            839,
            841,
            843,
            845,
            847,
            849,
            850,
            852,
            853,
            855,
            857,
            858,
            860,
            862,
            862,
            864,
            866,
            867,
            869,
            869,
            871,
            872,
            873,
            874,
            876,
            877,
            879,
            879,
            880,
            880,
            881,
            882,
            883,
            883,
            884,
            885,
            885,
            885,
            886,
            885,
            885,
            886,
            886,
            886,
            885,
            885,
            885,
            885,
            884,
            883,
            883,
            882,
            882,
            881,
            881,
            878,
            877,
            876,
            875,
            870,
            866,
            855,
            844,
            842,
            819,
            808,
            806,
            804,
            801,
            799,
            797,
            795,
            793,
            790,
            788,
            786,
            783,
            782,
            779,
            777,
            774,
            772,
            769,
            768,
            765,
            763,
            758,
            754,
            751,
            748,
            746,
            743,
            741,
            739,
            737,
            734,
            732,
            729,
            727,
            724,
            722,
            720,
            717,
            715,
            712,
            710,
            707,
            706,
            703,
            701,
            698,
            696,
            693,
            691,
            688,
            686,
            684,
            682,
            680,
            677,
            675,
            643,
            641,
            639,
            637,
            635,
            633,
            631,
            629,
            627,
            625,
            623,
            621,
            619,
            616,
            614,
            613,
            611,
            609,
            607,
            606,
            604,
            603,
            601,
            599,
            598,
            597,
            595,
            594,
            592,
            591,
            590,
            588,
            587,
            586,
            585,
            584,
            583,
            581,
            580,
            580,
            580,
            579,
            578,
            577,
            577,
            576,
            576,
            575,
            575,
            574,
            575,
            574,
            574,
            574,
            574,
            574,
            574,
            574,
            575,
            575,
            575,
            576,
            576,
            577,
        ],  # label_0_day_duration
        [
            876,
            884,
            884,
            883,
            879,
            878,
            873,
            873,
            872,
            870,
            868,
            865,
            864,
            862,
            861,
            859,
            858,
            856,
            853,
            851,
            849,
            848,
            846,
            840,
            838,
            837,
            835,
            833,
            831,
            829,
            827,
            825,
            823,
            821,
            817,
            814,
            812,
            811,
            760,
            755,
        ],
    ], # label_day_duration
    [[242.75, 229.78, 252.5, 298.45, 333.4, 356.55, 209.05, 185.54, 262.96, 233.9, 235.0, 287.78, 188.77, 162.54, 192.67, 192.03, 252.82, 282.34, 271.78, 244.94, 229.24, 237.99, 212.09, 249.86, 294.44, 271.97, 273.1, 259.41, 269.17, 190.93, 229.77, 191.19, 165.8, 227.0, 248.4, 238.13, 178.77, 179.15, 254.69, 238.74, 192.05, 150.55, 228.53, 285.41, 265.17, 216.91, 215.32, 195.15, 255.1, 235.1, 261.58, 216.75, 258.44, 222.24, 156.0, 157.27, 200.86, 172.11, 177.75, 198.37, 175.83, 219.24, 203.1, 154.87, 181.85, 105.78, 188.84, 232.1, 179.03, 173.35, 175.6, 246.89, 216.24, 146.38, 190.5, 125.76, 190.63, 203.4, 223.46, 296.93, 198.22, 166.97, 160.76, 78.35, 276.23, 138.4, 150.16, 226.22, 169.68, 190.64, 214.08, 218.88, 164.61, 177.1, 258.44, 185.56, 218.03, 196.48, 193.88, 140.71, 204.43, 168.72, 153.14, 178.87, 247.05, 185.92, 222.57, 216.94, 201.12, 171.69, 237.95, 134.76, 190.35, 120.84, 204.76, 155.79, 168.24, 90.49, 182.17, 194.89, 169.1, 220.64, 122.83, 171.73, 170.09, 177.93, 258.65, 173.25, 160.61, 177.16, 195.57, 198.41, 187.29, 220.05, 144.66, 210.26, 242.81, 112.47, 283.64, 110.64, 184.1, 199.91, 192.56, 180.53, 168.62, 173.79, 190.92, 177.46, 179.87, 196.94, 249.04, 152.52, 184.29, 199.95, 181.67, 207.61, 148.16, 148.72, 152.62, 213.73, 242.69, 139.79, 240.42, 156.0, 147.33, 225.75, 191.0, 147.98, 142.08, 157.97, 125.06, 196.54, 179.53, 159.61, 180.59, 207.58, 284.68, 253.76, 109.33, 200.97, 205.74, 239.05, 195.95, 127.74, 175.79, 336.16, 346.62, 322.66, 344.88, 366.29, 348.64, 244.62, 358.26, 249.62, 311.2, 322.93, 246.35, 239.25, 254.9, 353.97, 311.93, 264.52, 228.61, 192.72, 123.35, 177.11, 227.97, 217.09, 252.95, 196.14, 224.92, 166.68, 268.57, 257.55, 236.33, 187.65, 198.21, 249.11, 157.64, 191.64, 266.21, 169.5, 240.17, 237.6, 240.31, 254.19, 277.83, 278.73, 248.51, 243.99, 212.47, 204.47, 206.96, 193.44, 190.09, 173.46, 127.59, 186.33, 202.66, 229.75, 168.05, 169.41, 221.05, 196.05, 219.82, 264.07, 206.03, 213.77, 203.93, 233.73, 193.17, 197.07, 100.57, 228.14, 126.67, 143.77, 130.62, 216.23, 189.39, 187.34, 206.48, 180.7, 139.35, 179.7, 163.64, 211.62, 162.54, 228.11, 136.2, 180.85, 195.07, 208.74, 140.59, 128.02, 240.47, 173.75, 206.55, 165.41, 249.49, 207.39, 177.58, 189.13, 193.36, 174.32, 189.07, 214.47, 189.12, 216.78, 190.63, 221.78, 158.52, 196.41, 241.17, 220.05, 209.85, 188.84, 181.53, 213.03, 180.97, 234.53, 161.44, 151.11, 282.48, 197.52, 193.36, 126.89, 242.87, 196.02], [219.69, 207.85, 200.64, 207.27, 381.56, 321.2, 280.65, 209.48, 298.0, 328.98, 230.54, 200.17, 208.02, 239.63, 290.27, 287.92, 241.07, 251.93, 251.79, 229.82, 220.86, 222.36, 277.12, 357.17, 385.72, 341.63, 417.53, 499.36, 414.78, 345.36, 253.14, 322.87, 403.5, 310.42, 367.06, 346.66, 346.39, 311.98, 224.93, 219.41]] #power
]  


In [13]:
summeries = summerize_multiple_features(feature_set_1_data_2021)

summeries

[[{'model': 'gaussian',
   'mean': 96.74712643678161,
   'stdev': 46.818299432461835,
   'len': 348},
  {'model': 'gaussian',
   'mean': 99.39285714285714,
   'stdev': 48.671787314721584,
   'len': 308},
  {'model': 'gaussian',
   'mean': 76.375,
   'stdev': 19.338665268290217,
   'len': 40}],
 [{'model': 'gaussian',
   'mean': 1120.1666666666667,
   'stdev': 55.91008460053002,
   'len': 348},
  {'model': 'gaussian',
   'mean': 1112.474025974026,
   'stdev': 54.61958437611475,
   'len': 308},
  {'model': 'gaussian', 'mean': 1179.4, 'stdev': 16.2177490613027, 'len': 40}],
 [{'model': 'gaussian',
   'mean': 18.95402298850575,
   'stdev': 12.820784679261141,
   'len': 348},
  {'model': 'gaussian',
   'mean': 19.314935064935064,
   'stdev': 13.345624921899697,
   'len': 308},
  {'model': 'gaussian',
   'mean': 16.175,
   'stdev': 7.178627466841577,
   'len': 40}],
 [{'model': 'gaussian',
   'mean': 704.6149425287356,
   'stdev': 108.45618781430973,
   'len': 348},
  {'model': 'gaussian',
 

In [14]:
feature_set_1_data_2022 = [[84, 1044, 863, 577, 14, 262.41], [95, 1045, 862, 578, 23, 221.12], [78, 1045, 862, 578, 16, 297.36], [99, 1046, 861, 579, 20, 230.97], [134, 1047, 860, 580, 30, 240.86], [151, 1048, 859, 581, 35, 226.67], [153, 1049, 858, 582, 40, 212.26], [144, 1050, 857, 583, 38, 255.42], [178, 1051, 856, 584, 87, 189.84], [165, 1052, 855, 585, 66, 201.39], [91, 1053, 854, 586, 19, 102], [87, 1054, 853, 587, 18, 214], [99, 1055, 852, 588, 13, 215.98], [91, 1056, 850, 590, 15, 215.08], [119, 1057, 849, 591, 25, 228.03], [115, 1058, 848, 592, 25, 278.24], [78, 1059, 846, 594, 14, 283.82], [102, 1060, 845, 595, 20, 263.58], [115, 1061, 844, 596, 29, 272.3], [107, 1062, 842, 598, 28, 260.39], [105, 1063, 841, 599, 27, 225.52], [124, 1064, 839, 601, 33, 261.21], [153, 1065, 838, 602, 44, 215.5], [70, 1066, 836, 604, 15, 258.11], [127, 1067, 835, 605, 31, 178.08], [153, 1068, 833, 607, 43, 284.13], [154, 1069, 831, 609, 43, 217.96], [102, 1071, 829, 611, 22, 156.14], [93, 1072, 827, 613, 18, 238.32], [97, 1073, 825, 615, 20, 247.06], [89, 1074, 823, 617, 19, 206.92], [119, 1075, 822, 618, 35, 220.02], [91, 1076, 820, 620, 24, 196.5], [74, 1077, 818, 622, 15, 192.45], [72, 1078, 816, 624, 13, 219.05], [89, 1079, 814, 626, 12, 259.67], [76, 1081, 811, 629, 12, 161.6], [84, 1082, 809, 631, 16, 261.4], [87, 1083, 807, 633, 19, 267.23], [97, 1084, 805, 635, 23, 182.21], [124, 1085, 803, 637, 32, 183.26], [161, 1086, 801, 639, 59, 270.53], [168, 1087, 799, 641, 71, 223.4], [153, 1088, 797, 643, 41, 269.58], [112, 1089, 795, 645, 30, 274.78], [102, 1090, 793, 647, 26, 222.47], [87, 1091, 790, 650, 22, 292.95], [91, 1092, 788, 652, 21, 285.64], [115, 1094, 785, 655, 28, 277.46], [119, 1095, 783, 657, 30, 266.01], [91, 1096, 781, 659, 17, 122.76], [84, 1097, 778, 662, 15, 210.64], [93, 1098, 776, 664, 16, 158.94], [84, 1099, 774, 666, 13, 218.04], [84, 1100, 772, 668, 16, 131], [152, 1101, 769, 671, 42, 294.27], [151, 1102, 767, 673, 39, 284.21], [132, 1103, 765, 675, 25, 165.71], [117, 1104, 762, 678, 17, 281.62], [99, 1106, 757, 683, 22, 192.85], [82, 1107, 755, 685, 14, 233.83], [127, 1108, 753, 687, 23, 200.53], [161, 1109, 750, 690, 32, 172.33], [175, 1110, 748, 692, 16, 232.57], [127, 1111, 745, 695, 10, 213], [112, 1112, 743, 697, 19, 244.53], [134, 1113, 741, 699, 24, 256.05], [154, 1114, 738, 702, 35, 113.36], [158, 1115, 736, 704, 46, 260.07], [157, 1116, 733, 707, 36, 179.49], [156, 1116, 732, 708, 38, 112.36], [57, 1117, 729, 711, 11, 234.67], [55, 1118, 727, 713, 9, 158.3], [110, 1119, 724, 716, 29, 211.27], [156, 1120, 722, 718, 38, 178.41], [142, 1121, 719, 721, 26, 164.95], [27, 1122, 717, 723, 3, 152], [22, 1123, 714, 726, 4, 249.7], [66, 1124, 712, 728, 14, 237.17], [84, 1125, 709, 731, 17, 253.94], [99, 1126, 707, 733, 25, 260.09], [95, 1127, 704, 736, 21, 245.4], [112, 1128, 702, 738, 26, 225.33], [154, 1129, 699, 741, 42, 168.71], [46, 1129, 698, 742, 7, 222.78], [93, 1130, 695, 745, 14, 210.3], [91, 1131, 693, 747, 13, 183.56], [110, 1132, 690, 750, 17, 159.08], [102, 1133, 688, 752, 17, 206.26], [66, 1134, 685, 755, 11, 213.31], [59, 1135, 683, 757, 10, 235.03], [91, 1136, 680, 760, 19, 237.23], [119, 1137, 678, 762, 28, 144.19], [137, 1138, 675, 765, 35, 238.28], [117, 1138, 674, 766, 27, 211.38], [127, 1139, 671, 769, 22, 180.74], [102, 1140, 669, 771, 11, 201.73], [124, 1141, 666, 774, 18, 230.27], [139, 1142, 664, 776, 23, 187.91], [124, 1143, 661, 779, 24, 242.48], [107, 1144, 659, 781, 20, 248.62], [89, 1145, 656, 784, 19, 268.05], [53, 1146, 654, 786, 9, 174.8], [50, 1147, 652, 788, 7, 159.15], [74, 1148, 649, 791, 12, 269.07], [107, 1148, 648, 792, 21, 211.72], [134, 1149, 646, 794, 29, 211.39], [149, 1150, 643, 797, 34, 137.36], [155, 1151, 641, 799, 39, 276.83], [155, 1152, 638, 802, 43, 172.32], [142, 1153, 636, 804, 32, 192.17], [129, 1154, 634, 806, 26, 261.31], [152, 1155, 632, 808, 38, 144.82], [142, 1156, 629, 811, 31, 188.41], [134, 1157, 627, 813, 33, 217.43], [124, 1158, 625, 815, 22, 163.07], [210, 1158, 624, 816, 17, 188.82], [162, 1159, 621, 819, 17, 177.85], [42, 1160, 619, 821, 5, 249.77], [59, 1161, 617, 823, 10, 171.3], [119, 1162, 615, 825, 16, 112.78], [129, 1163, 613, 827, 23, 196.45], [107, 1164, 611, 829, 22, 229.33], [112, 1165, 608, 832, 22, 181.25], [99, 1166, 606, 834, 19, 204.78], [107, 1167, 604, 836, 22, 219.01], [89, 1168, 602, 838, 14, 181.85], [66, 1168, 601, 839, 9, 155.38], [61, 1169, 599, 841, 11, 192.56], [80, 1170, 597, 843, 15, 111.98], [91, 1171, 595, 845, 18, 89.93], [95, 1172, 593, 847, 17, 202.39], [91, 1173, 591, 849, 15, 191.23], [97, 1174, 590, 850, 13, 183.45], [78, 1175, 588, 852, 11, 137.85], [93, 1175, 587, 853, 15, 183.45], [137, 1176, 585, 855, 28, 210.82], [153, 1177, 583, 857, 34, 165.68], [122, 1178, 582, 858, 22, 203.57], [102, 1179, 580, 860, 24, 224.74], [97, 1180, 578, 862, 22, 181.56], [110, 1180, 578, 862, 27, 161.74], [157, 1181, 576, 864, 46, 148.92], [93, 1182, 574, 866, 19, 193.5], [80, 1183, 573, 867, 15, 197.24], [82, 1184, 571, 869, 10, 216.75], [76, 1184, 571, 869, 11, 154.83], [105, 1185, 569, 871, 21, 157.62], [104, 1186, 568, 872, 22, 125.56], [89, 1186, 567, 873, 19, 160.47], [82, 1187, 566, 874, 10, 163.97], [80, 1188, 564, 876, 11, 185.94], [84, 1188, 564, 876, 16, 233.17], [102, 1190, 561, 879, 21, 187.25], [78, 1190, 561, 879, 17, 260.73], [33, 1191, 560, 880, 4, 216.54], [35, 1191, 560, 880, 3, 148.87], [59, 1192, 559, 881, 10, 190.86], [78, 1192, 558, 882, 15, 195.08], [80, 1193, 557, 883, 18, 119.55], [72, 1194, 556, 884, 15, 107.73], [76, 1194, 556, 884, 18, 142.11], [53, 1195, 555, 885, 6, 204.25], [46, 1195, 555, 885, 7, 194.85], [57, 1195, 555, 885, 9, 235.33], [97, 1196, 554, 886, 23, 225.54], [74, 1196, 555, 885, 13, 144.62], [80, 1196, 555, 885, 16, 182.36], [78, 1197, 554, 886, 16, 238.33], [84, 1197, 555, 885, 20, 224.2], [63, 1197, 555, 885, 13, 270.34], [129, 1197, 555, 885, 36, 161.45], [78, 1197, 555, 885, 15, 202.32], [55, 1197, 556, 884, 7, 172.19], [61, 1197, 556, 884, 8, 229.53], [53, 1197, 557, 883, 7, 146.64], [30, 1197, 557, 883, 4, 219.91], [76, 1196, 562, 878, 17, 245.16], [76, 1196, 563, 877, 17, 194.17], [53, 1194, 567, 873, 6, 250.58], [59, 1194, 567, 873, 11, 361.06], [91, 1193, 570, 870, 24, 187.49], [95, 1190, 576, 864, 22, 173.09], [55, 1189, 578, 862, 8, 297.62], [82, 1189, 579, 861, 19, 198.3], [102, 1188, 581, 859, 26, 230.62], [63, 1187, 582, 858, 13, 202.4], [46, 1179, 598, 842, 6, 394.97], [70, 1173, 609, 831, 12, 299.54], [38, 1172, 611, 829, 4, 137.37], [53, 1169, 615, 825, 8, 333.24], [89, 1167, 619, 821, 18, 313.49], [74, 1159, 632, 808, 13, 377], [61, 1153, 643, 797, 6, 236.77], [66, 1151, 645, 795, 7, 248.02], [91, 1150, 647, 793, 20, 245.38], [68, 1148, 650, 790, 9, 303.37], [61, 1147, 652, 788, 8, 365.53], [53, 1146, 654, 786, 5, 282.78], [33, 1144, 657, 783, 3, 227.31], [42, 1143, 658, 782, 4, 314.55], [57, 1140, 663, 777, 6, 225.37], [53, 1138, 666, 774, 3, 249.66], [42, 1137, 668, 772, 4, 268.8], [38, 1135, 671, 769, 1, 262.63], [42, 1134, 672, 768, 5, 180.41], [84, 1132, 675, 765, 14, 231.51], [46, 1128, 682, 758, 4, 174.84], [72, 1126, 685, 755, 11, 175.31], [70, 1125, 686, 754, 12, 286.24], [72, 1123, 689, 751, 12, 209.06], [68, 1121, 692, 748, 8, 247.03], [72, 1120, 694, 746, 9, 236.05], [76, 1118, 697, 743, 12, 174.89], [68, 1114, 703, 737, 7, 240.68], [87, 1112, 706, 734, 6, 234.43], [89, 1111, 708, 732, 11, 278.38], [70, 1109, 711, 729, 8, 215.47], [80, 1108, 713, 727, 9, 139.84], [68, 1106, 716, 724, 7, 208.74], [80, 1104, 718, 722, 10, 156.47], [84, 1103, 720, 720, 13, 275.09], [87, 1101, 723, 717, 15, 227.04], [89, 1100, 725, 715, 17, 294.99], [142, 1098, 728, 712, 38, 251.16], [132, 1097, 730, 710, 34, 292.22], [137, 1095, 733, 707, 34, 243.63], [91, 1094, 734, 706, 22, 162.27], [57, 1092, 737, 703, 9, 226.35], [57, 1091, 739, 701, 7, 182.76], [66, 1089, 742, 698, 8, 244.51], [33, 1088, 744, 696, 2, 175.29], [46, 1086, 747, 693, 4, 146.38], [42, 1085, 749, 691, 4, 143.49], [42, 1083, 752, 688, 4, 195.3], [46, 1082, 754, 686, 3, 148.55], [63, 1080, 756, 684, 8, 174.85], [74, 1079, 758, 682, 12, 122.21], [76, 1078, 760, 680, 10, 238.59], [84, 1076, 763, 677, 14, 191.06], [93, 1075, 765, 675, 18, 149.49], [99, 1074, 767, 673, 21, 182.83], [93, 1072, 770, 670, 10, 235.61], [78, 1071, 772, 668, 9, 132.86], [84, 1069, 775, 665, 11, 191.92], [102, 1068, 777, 663, 16, 168.89], [132, 1067, 779, 661, 28, 318.78], [115, 1066, 781, 659, 27, 197.65], [110, 1064, 784, 656, 19, 191.57], [84, 1063, 786, 654, 11, 190.92], [59, 1062, 788, 652, 7, 209.79], [63, 1061, 790, 650, 7, 163.32], [97, 1059, 793, 647, 18, 258.29], [115, 1058, 795, 645, 23, 116.51], [80, 1057, 797, 643, 15, 166.09], [78, 1056, 799, 641, 17, 100.22], [78, 1055, 801, 639, 14, 76.46], [127, 1054, 803, 637, 27, 180.31], [97, 1053, 805, 635, 13, 229.82], [95, 1052, 807, 633, 14, 293.4], [82, 1051, 809, 631, 8, 132.62], [87, 1050, 811, 629, 13, 216.27], [80, 1049, 813, 627, 14, 245.65], [99, 1048, 815, 625, 19, 217.95], [124, 1047, 817, 623, 29, 151.11], [154, 1046, 819, 621, 43, 166.89], [161, 1045, 821, 619, 52, 204.19], [142, 1044, 824, 616, 33, 175.55], [127, 1043, 826, 614, 31, 198.88], [66, 1043, 827, 613, 11, 198.36], [74, 1042, 829, 611, 15, 159.41], [89, 1041, 831, 609, 15, 163.27], [84, 1040, 833, 607, 16, 199.75], [99, 1040, 834, 606, 18, 181.01], [119, 1039, 836, 604, 25, 210.57], [93, 1039, 837, 603, 17, 293.13], [89, 1038, 839, 601, 18, 210.75], [107, 1037, 841, 599, 23, 176.27], [139, 1037, 842, 598, 33, 138.6], [124, 1037, 843, 597, 27, 107.08], [139, 1036, 845, 595, 28, 223.55], [144, 1036, 846, 594, 31, 204.81], [155, 1035, 848, 592, 18, 117.89], [72, 1035, 849, 591, 11, 171.09], [84, 1035, 850, 590, 14, 113.69], [72, 1034, 852, 588, 11, 195.66], [84, 1034, 853, 587, 7, 181.05], [72, 1034, 854, 586, 9, 209.48], [68, 1034, 855, 585, 9, 186.71], [91, 1034, 856, 584, 20, 187.43], [63, 1034, 857, 583, 9, 181.34], [70, 1033, 859, 581, 12, 169.91], [82, 1033, 860, 580, 16, 205.43], [99, 1033, 860, 580, 16, 202.58], [82, 1034, 860, 580, 11, 165.07], [115, 1034, 861, 579, 24, 269.77], [107, 1034, 862, 578, 22, 237.22], [119, 1034, 863, 577, 26, 254.77], [91, 1034, 863, 577, 20, 205.14], [263, 1034, 864, 576, 29, 236.22], [137, 1035, 864, 576, 11, 168.11], [91, 1035, 865, 575, 18, 235.58], [91, 1035, 865, 575, 19, 170.96], [74, 1035, 866, 574, 15, 151.75], [68, 1036, 865, 575, 10, 200.37], [68, 1036, 866, 574, 12, 194.4], [89, 1037, 866, 574, 21, 249.31], [99, 1037, 866, 574, 25, 222.43], [91, 1038, 866, 574, 17, 297.52], [70, 1038, 866, 574, 12, 156.8], [82, 1039, 866, 574, 18, 208.7], [105, 1039, 866, 574, 29, 222.28], [129, 1040, 865, 575, 36, 180.44], [127, 1041, 865, 575, 35, 226.43], [95, 1041, 865, 575, 22, 197.53], [84, 1042, 864, 576, 19, 181.51], [95, 1043, 864, 576, 25, 271.72], [119, 1044, 863, 577, 34, 260.82], [119, 1189, 563, 877, 26, 222.66],[78, 1193, 557, 883, 15, 220.81],[76, 1197, 554, 886, 15, 209.68],[82, 1197, 554, 886, 15, 287.07],[66, 1197, 557, 883, 9, 268.05],[89, 1197, 558, 882, 17, 236.13],[115, 1197, 558, 882, 32, 351.73],[110, 1197, 559, 881, 30, 322.73],[102, 1197, 559, 881, 28, 223.58],[95, 1196, 561, 879, 26, 278.09],[74, 1196, 562, 878, 15, 227.16],[127, 1195, 564, 876, 33, 231.91],[80, 1195, 565, 875, 18, 276.19],[53, 1194, 568, 872, 8, 284.02],[80, 1193, 570, 870, 16, 215.66],[78, 1192, 572, 868, 18, 222.18],[82, 1191, 574, 866, 18, 231.2],[97, 1191, 575, 865, 23, 211.53],[68, 1186, 584, 856, 10, 294.28],[95, 1186, 585, 855, 21, 295.21],[97, 1185, 587, 853, 24, 261.91],[76, 1184, 589, 851, 16, 247.58],[76, 1183, 591, 849, 17, 266.08],[66, 1182, 592, 848, 11, 286.6],[50, 1181, 594, 846, 6, 298.03],[59, 1180, 596, 844, 10, 206.63],[72, 1178, 600, 840, 18, 301.76],[66, 1177, 602, 838, 11, 282.65],[68, 1176, 603, 837, 11, 222.11],[105, 1175, 605, 835, 26, 338.41],[89, 1174, 607, 833, 20, 282.77],[53, 1171, 613, 827, 7, 291.12],[78, 1168, 617, 823, 16, 301.75],[72, 1166, 621, 819, 15, 276.19],[76, 1165, 623, 817, 12, 315.31],[76, 1163, 626, 814, 12, 289.85],[78, 1162, 628, 812, 15, 232.94],[76, 1161, 629, 811, 14, 345.68],[82, 1158, 634, 806, 16, 262.5],[80, 1157, 636, 804, 17, 257.76],[76, 1155, 639, 801, 16, 306.01],[63, 1154, 641, 799, 9, 288.44],[68, 1141, 661, 779, 9, 251.1],[89, 1131, 677, 763, 14, 263.15],[70, 1129, 680, 760, 13, 226.4],[72, 1117, 699, 741, 15, 247.14],[72, 1115, 701, 739, 14, 277.89]]

In [17]:
# label_0_cloud,label_1_cloud,label_0_wind,label_1_wind,label_0_humidity,label_1_humidity
feature_set_2_data_2021 = [
    [
        [
            1.3,
            0.0,
            0.0,
            1.9,
            3.6,
            5.8,
            5.6,
            3.0,
            3.3,
            1.4,
            1.4,
            1.8,
            10.0,
            9.8,
            10.0,
            2.8,
            8.8,
            1.1,
            6.3,
            0.3,
            2.0,
            7.5,
            5.3,
            1.0,
            1.6,
            8.5,
            0.0,
            1.0,
            5.5,
            3.0,
            0.9,
            2.3,
            3.6,
            3.4,
            0.3,
            6.5,
            6.1,
            3.4,
            6.9,
            7.3,
            1.6,
            3.9,
            5.4,
            5.0,
            9.9,
            3.9,
            0.0,
            0.1,
            5.5,
            2.5,
            1.8,
            0.0,
            0.0,
            0.0,
            5.5,
            5.4,
            0.0,
            3.8,
            4.8,
            0.0,
            1.1,
            2.1,
            7.0,
            5.1,
            0.0,
            5.9,
            5.9,
            3.1,
            3.4,
            6.4,
            5.5,
            8.0,
            1.9,
            0.4,
            2.8,
            1.1,
            6.4,
            5.6,
            8.0,
            4.1,
            0.8,
            1.1,
            2.9,
            0.3,
            1.1,
            0.0,
            1.8,
            0.4,
            4.1,
            9.1,
            10.0,
            9.8,
            1.5,
            0.4,
            0.0,
            2.6,
            6.5,
            0.8,
            5.1,
            8.1,
            7.4,
            6.9,
            6.1,
            6.4,
            2.3,
            9.5,
            3.8,
            7.6,
            8.0,
            5.8,
            9.4,
            4.5,
            0.5,
            6.4,
            9.6,
            6.1,
            3.0,
            6.6,
            4.5,
            7.9,
            10.0,
            9.3,
            7.1,
            2.4,
            0.9,
            2.5,
            2.9,
            5.0,
            3.6,
            1.6,
            2.0,
            2.1,
            4.6,
            7.1,
            2.4,
            5.0,
            9.0,
            6.3,
            0.6,
            7.0,
            5.4,
            7.0,
            9.6,
            10.0,
            6.8,
            6.5,
            6.5,
            8.3,
            2.1,
            3.3,
            4.4,
            6.1,
            8.9,
            8.4,
            4.6,
            6.9,
            6.1,
            6.6,
            4.1,
            2.9,
            0.9,
            4.4,
            3.4,
            3.3,
            8.8,
            9.4,
            6.3,
            5.0,
            1.5,
            5.8,
            10.0,
            8.1,
            10.0,
            7.4,
            6.6,
            4.9,
            9.8,
            5.9,
            4.1,
            8.8,
            7.3,
            9.4,
            9.3,
            8.0,
            7.6,
            4.1,
            4.0,
            4.0,
            3.9,
            7.3,
            6.6,
            9.0,
            9.6,
            9.0,
            5.0,
            6.4,
            8.9,
            10.0,
            9.4,
            9.8,
            6.0,
            0.5,
            4.6,
            8.1,
            9.1,
            6.4,
            1.1,
            0.0,
            0.4,
            6.1,
            7.4,
            9.9,
            7.6,
            9.8,
            9.9,
            8.0,
            3.8,
            8.6,
            9.4,
            7.1,
            2.0,
            4.6,
            5.4,
            9.6,
            10.0,
            9.1,
            5.0,
            7.1,
            6.1,
            0.1,
            2.0,
            4.9,
            5.8,
            2.3,
            5.6,
            7.8,
            8.4,
            6.0,
            0.0,
            0.5,
            4.1,
            7.6,
            5.5,
            2.0,
            0.0,
            1.1,
            5.8,
            9.0,
            4.6,
            7.9,
            5.6,
            9.1,
            9.4,
            1.6,
            2.6,
            5.6,
            1.1,
            1.9,
            0.0,
            0.0,
            2.3,
            4.0,
            9.0,
            4.3,
            0.0,
            2.6,
            0.0,
            0.0,
            4.0,
            4.4,
            0.3,
            1.8,
            8.9,
            6.8,
            2.3,
            4.4,
            0.0,
            0.1,
            4.0,
            0.0,
            2.5,
            3.8,
            6.3,
            1.9,
            1.4,
            8.3,
            5.6,
            10.0,
            4.9,
            6.3,
            10.0,
            10.0,
            5.6,
            0.0,
            6.9,
            6.9,
            2.6,
            0.0,
            0.0,
            1.4,
            5.0,
            6.5,
            2.9,
            0.5,
            3.1,
            4.8,
            9.5,
            5.8,
        ],  # label_0_cloud
        [
            3.5,
            3.9,
            6.5,
            1.4,
            6.3,
            7.6,
            9.4,
            10.0,
            10.0,
            9.9,
            9.0,
            2.3,
            2.9,
            2.3,
            6.6,
            10.0,
            7.3,
            7.1,
            5.9,
            3.3,
            4.4,
            4.8,
            6.4,
            3.9,
            4.3,
            2.8,
            5.8,
            4.4,
            2.4,
            8.0,
            10.0,
            10.0,
            8.9,
            8.3,
            6.5,
            4.5,
            4.6,
            3.1,
            3.3,
            6.5,
        ],
    ],  # label_0_cloud
    [
        [
            2.7,
            2.5,
            1.8,
            1.9,
            1.6,
            2.2,
            2.2,
            1.7,
            2.3,
            1.5,
            1.6,
            2.2,
            2.5,
            2.5,
            2.8,
            2.7,
            1.8,
            2.0,
            2.5,
            3.7,
            1.8,
            2.6,
            2.7,
            4.6,
            2.1,
            2.5,
            3.5,
            2.4,
            2.5,
            2.2,
            2.0,
            1.8,
            1.6,
            1.5,
            2.5,
            2.5,
            2.2,
            1.8,
            1.8,
            2.3,
            2.7,
            2.5,
            2.7,
            1.8,
            1.5,
            3.3,
            3.3,
            2.1,
            2.2,
            1.9,
            3.0,
            2.9,
            2.1,
            1.9,
            2.0,
            3.1,
            2.8,
            2.1,
            1.9,
            1.7,
            2.1,
            2.3,
            2.0,
            2.3,
            2.1,
            2.4,
            3.7,
            3.2,
            2.9,
            3.0,
            2.4,
            1.7,
            2.6,
            2.1,
            1.6,
            1.7,
            1.8,
            2.5,
            2.3,
            2.3,
            3.6,
            2.3,
            2.1,
            1.9,
            2.2,
            2.1,
            1.8,
            2.4,
            2.3,
            2.6,
            3.4,
            2.5,
            2.9,
            3.2,
            2.5,
            2.7,
            3.6,
            2.7,
            1.8,
            1.6,
            2.2,
            2.5,
            2.0,
            2.7,
            2.4,
            2.4,
            2.5,
            2.6,
            2.1,
            2.7,
            1.9,
            3.1,
            1.9,
            2.1,
            2.9,
            3.3,
            2.5,
            2.1,
            2.1,
            2.1,
            2.5,
            5.4,
            2.6,
            2.2,
            2.3,
            1.8,
            2.3,
            1.8,
            2.0,
            2.0,
            2.4,
            2.5,
            3.4,
            2.1,
            2.3,
            3.0,
            3.0,
            1.9,
            2.2,
            2.0,
            2.4,
            2.1,
            1.7,
            1.8,
            1.6,
            2.2,
            2.5,
            1.9,
            2.7,
            2.4,
            2.6,
            2.6,
            2.1,
            2.4,
            2.0,
            2.3,
            2.2,
            1.7,
            1.5,
            2.1,
            2.1,
            2.0,
            2.0,
            2.2,
            1.9,
            2.1,
            2.2,
            1.5,
            2.0,
            2.1,
            2.9,
            2.5,
            2.3,
            1.8,
            1.9,
            1.8,
            2.3,
            3.7,
            2.3,
            2.0,
            2.3,
            1.9,
            2.3,
            1.4,
            2.2,
            1.9,
            2.1,
            1.6,
            1.9,
            1.6,
            2.4,
            2.4,
            2.2,
            2.8,
            2.6,
            1.8,
            1.8,
            3.3,
            2.1,
            1.1,
            1.3,
            2.0,
            1.6,
            2.2,
            1.8,
            1.8,
            2.4,
            2.5,
            2.3,
            1.6,
            1.5,
            1.8,
            2.4,
            2.6,
            1.7,
            1.4,
            1.9,
            1.6,
            2.7,
            2.3,
            1.7,
            2.5,
            2.9,
            2.3,
            2.4,
            1.8,
            2.0,
            1.4,
            2.2,
            2.4,
            1.7,
            1.7,
            2.1,
            2.6,
            1.6,
            2.1,
            2.5,
            1.9,
            1.6,
            1.7,
            1.5,
            1.5,
            1.4,
            2.0,
            1.7,
            1.7,
            1.8,
            2.4,
            2.9,
            2.8,
            2.6,
            3.3,
            4.9,
            3.2,
            1.7,
            2.1,
            3.9,
            2.2,
            2.2,
            2.3,
            1.7,
            1.5,
            1.9,
            2.9,
            3.5,
            2.4,
            1.5,
            1.7,
            1.8,
            3.3,
            3.8,
            1.8,
            1.4,
            2.4,
            3.2,
            2.5,
            2.8,
            2.0,
            2.4,
            1.7,
            1.6,
            2.0,
            2.3,
            2.6,
            3.0,
            2.0,
            0.0,
            0.0,
            2.7,
            2.4,
            4.5,
            3.6,
            5.2,
            3.2,
            2.2,
            1.9,
            3.4,
            3.0,
            2.7,
            1.8,
            1.6,
            1.5,
            1.8,
            1.4,
            1.6,
            1.7,
            2.6,
            2.0,
        ],  # label_0_wind
        [
            2.3,
            2.2,
            3.2,
            2.0,
            2.7,
            2.2,
            1.5,
            1.6,
            2.9,
            3.0,
            1.7,
            1.7,
            1.8,
            1.8,
            2.6,
            2.1,
            2.0,
            2.7,
            1.9,
            2.1,
            2.0,
            1.9,
            1.7,
            2.1,
            1.9,
            2.0,
            1.7,
            2.7,
            2.9,
            3.2,
            3.7,
            2.3,
            1.9,
            1.7,
            2.3,
            2.1,
            1.7,
            1.9,
            1.8,
            1.9,
        ],
    ],  # label_1_wind
    [
        [
            64.87,
            38.43,
            44.39,
            48.91,
            53.43,
            53.87,
            50.3,
            44.3,
            44.91,
            53.57,
            62.22,
            77.57,
            82.78,
            47.61,
            65.57,
            58.91,
            55.65,
            66.57,
            38.43,
            38.35,
            59.87,
            93.17,
            67.17,
            62.91,
            61.78,
            79.61,
            48.83,
            53.65,
            44.52,
            61.43,
            50.83,
            70.78,
            79.74,
            68.04,
            43.83,
            49.83,
            74.09,
            71.96,
            61.35,
            50.78,
            46.13,
            65.7,
            60.0,
            44.3,
            44.17,
            64.43,
            60.7,
            66.09,
            60.26,
            33.7,
            32.17,
            36.0,
            33.57,
            48.96,
            49.22,
            89.26,
            75.57,
            65.52,
            62.0,
            69.13,
            70.61,
            61.7,
            59.48,
            56.52,
            62.22,
            52.52,
            55.39,
            62.26,
            71.39,
            66.0,
            53.7,
            44.83,
            50.74,
            47.78,
            72.65,
            68.52,
            53.87,
            63.61,
            60.26,
            67.22,
            60.87,
            68.57,
            95.78,
            63.26,
            47.78,
            40.78,
            33.48,
            35.48,
            78.87,
            67.96,
            47.96,
            49.57,
            49.7,
            34.7,
            38.52,
            48.91,
            45.78,
            67.52,
            75.43,
            56.96,
            61.78,
            59.61,
            57.0,
            63.09,
            65.83,
            56.52,
            43.96,
            37.87,
            42.74,
            40.74,
            51.04,
            49.96,
            60.35,
            66.26,
            50.96,
            82.26,
            80.04,
            68.91,
            57.96,
            80.39,
            65.13,
            47.65,
            59.13,
            66.3,
            51.09,
            67.78,
            51.17,
            37.48,
            53.57,
            59.87,
            78.09,
            97.43,
            92.74,
            73.39,
            52.22,
            56.57,
            86.43,
            70.04,
            65.0,
            58.83,
            70.65,
            60.17,
            81.7,
            86.74,
            78.43,
            71.91,
            79.7,
            74.96,
            80.61,
            69.87,
            66.83,
            70.17,
            71.48,
            69.39,
            59.0,
            65.13,
            84.09,
            74.83,
            69.83,
            79.57,
            60.09,
            70.22,
            91.48,
            73.52,
            72.87,
            67.83,
            65.74,
            70.61,
            71.78,
            67.96,
            78.57,
            77.0,
            78.74,
            79.04,
            64.0,
            77.74,
            97.35,
            80.65,
            77.17,
            77.43,
            78.61,
            86.57,
            67.78,
            61.96,
            54.91,
            81.48,
            83.09,
            64.35,
            57.87,
            61.96,
            87.48,
            81.52,
            89.3,
            94.04,
            86.22,
            85.39,
            83.61,
            80.61,
            81.83,
            77.96,
            91.87,
            84.48,
            70.13,
            60.91,
            57.43,
            60.39,
            67.3,
            98.17,
            81.43,
            75.52,
            64.0,
            67.48,
            63.35,
            57.48,
            60.57,
            61.7,
            64.39,
            63.74,
            63.57,
            83.09,
            79.7,
            67.13,
            67.57,
            72.74,
            63.96,
            65.52,
            73.7,
            93.65,
            83.78,
            74.91,
            76.96,
            70.39,
            79.7,
            82.0,
            92.22,
            85.3,
            91.26,
            83.17,
            89.35,
            79.17,
            66.7,
            65.87,
            73.57,
            74.48,
            54.57,
            61.7,
            71.91,
            77.09,
            75.57,
            70.35,
            79.65,
            69.26,
            61.17,
            57.74,
            80.87,
            81.74,
            72.96,
            61.04,
            55.96,
            65.26,
            68.83,
            71.26,
            66.22,
            63.91,
            69.7,
            84.09,
            78.39,
            73.22,
            63.04,
            60.43,
            63.43,
            55.87,
            56.61,
            53.26,
            52.61,
            59.78,
            81.87,
            51.09,
            66.52,
            53.65,
            49.57,
            49.04,
            59.87,
            66.91,
            69.74,
            70.17,
            80.09,
            83.96,
            63.74,
            43.43,
            62.13,
            85.52,
            71.74,
            50.04,
            60.43,
            64.22,
            76.17,
            73.39,
            46.35,
            54.17,
            67.0,
            42.3,
            42.83,
            58.13,
            73.83,
            73.35,
            48.96,
            35.22,
        ],  # label_0_humidity
        [
            64.78,
            72.3,
            72.57,
            70.91,
            74.09,
            83.13,
            75.35,
            75.96,
            71.26,
            70.87,
            63.48,
            77.78,
            76.7,
            64.83,
            59.65,
            61.09,
            58.3,
            55.74,
            60.13,
            66.96,
            70.04,
            62.83,
            70.52,
            77.35,
            70.35,
            67.87,
            68.83,
            66.0,
            75.61,
            62.83,
            62.17,
            70.09,
            67.74,
            70.83,
            59.35,
            60.96,
            63.52,
            70.7,
            76.61,
            73.13,
        ],
    ],
]  # label_1_humidity


In [18]:
feature_set_2_data_2022 = [[1.4, 1.5, 44.39], [4.1, 2.3, 59.04], [4.9, 1.8, 56.3], [0, 2.4, 53.87], [7.3, 1.7, 46.43], [0, 2.3, 46.78], [0, 1.5, 46.87], [5.9, 1.7, 62.65], [3.1, 1.7, 72.13], [7, 2, 66.22], [0.4, 4, 56], [2.8, 2.4, 48.74], [2.3, 3.3, 45], [2.6, 1.7, 41.87], [8.5, 1.9, 47.43], [1.6, 3.3, 54.57], [2.5, 2.2, 58.61], [0.3, 2.3, 47.48], [5.1, 2.1, 71.09], [0, 1.8, 64.22], [0, 1.8, 55.57], [5.9, 1.7, 52.39], [4.8, 1.7, 50.83], [4.9, 3.5, 56.22], [9.4, 2.3, 79.43], [2.4, 2.2, 64.35], [2.1, 2.1, 49.83], [2, 2.8, 50.87], [0.8, 2.4, 48.57], [0.6, 2.2, 44.48], [3.3, 1.9, 56.87], [6.6, 2.6, 76.74], [3.5, 2.4, 61.74], [1.5, 2.4, 52.61], [1.9, 3.1, 52.22], [0.9, 3.1, 48.22], [1, 2.5, 50.74], [6.1, 1.8, 50.65], [1.5, 1.9, 53.09], [5.5, 1.8, 53.04], [1.6, 2.3, 55.3], [2.6, 1.8, 61.09], [3.5, 1.7, 52.43], [5.3, 1.8, 41.65], [6.6, 2.9, 69.22], [5.6, 3.8, 57.96], [0.9, 3.7, 49.78], [0, 2.5, 41.83], [3.9, 2, 40.43], [6.5, 2.9, 52.48], [0.1, 4.1, 41.26], [2.5, 2.9, 54.04], [1, 3.2, 42.87], [0, 2.6, 38.83], [0, 2.2, 49.35], [3.8, 2.3, 76.57], [8, 3.9, 79.43], [0, 2.8, 56.35], [8, 1.5, 62.78], [6.1, 2.3, 71.61], [1, 2.7, 54.52], [0, 2.1, 54.43], [5.1, 3.6, 59.17], [0.1, 4.3, 38.65], [1.3, 2.6, 38.43], [6.1, 2, 45.52], [4.9, 2, 53.61], [2.3, 1.9, 68.74], [6.6, 2.1, 51.3], [5.3, 1.9, 57.65], [5.8, 2.7, 70.26], [9.4, 1.9, 95.83], [6.4, 3.2, 90.61], [3.8, 2, 72.7], [3.8, 2.1, 63.26], [10, 2.7, 62.13], [9.1, 4.4, 61.22], [8.9, 3, 84.43], [5.1, 2.6, 67.48], [7.5, 2.4, 49.35], [2.5, 2.2, 48.87], [9, 2.3, 57.83], [8.4, 1.8, 70.35], [7.9, 2.6, 67.74], [8.3, 2.9, 84.74], [0.3, 3, 50.78], [0.5, 2.1, 35.17], [1.4, 2.1, 50.78], [7, 1.8, 76], [7, 2.5, 65.3], [1.1, 2.6, 51.35], [0.5, 2.3, 53.87], [0, 2.1, 55.35], [0, 2.3, 50.96], [6.1, 2.4, 40.74], [6.4, 2.5, 52.17], [2.4, 3.3, 42], [0, 2.1, 50.43], [2.4, 3, 43.17], [1.5, 2, 45.74], [2.8, 2.4, 58.61], [8.6, 2.8, 76.74], [10, 4.4, 71.7], [9.3, 4, 52.74], [1, 3, 47.61], [0.3, 2.5, 46.61], [2.6, 2.3, 54.7], [4.8, 2.4, 46.17], [0, 2.4, 51.65], [0, 2.3, 49.17], [5.3, 2.8, 47.17], [8.5, 3.5, 75.96], [6, 1.7, 58.43], [6.4, 2.3, 70.65], [10, 1.4, 81.09], [5, 3.1, 61.57], [0.4, 2.7, 30.43], [5.8, 2.6, 43.26], [8.8, 3.4, 71.57], [8, 2.9, 67.22], [1.3, 3.1, 53.57], [2, 2.4, 57.96], [1, 2.5, 47.13], [0, 2.7, 43.83], [0.5, 2.2, 46.61], [5.3, 2.5, 66.17], [8.6, 3, 55.57], [9.1, 2.8, 49], [3.1, 2.2, 56.96], [7.3, 2.2, 39.43], [6.9, 2.4, 48.83], [3.9, 1.9, 52.43], [5.9, 2.2, 44.83], [4.4, 3.3, 45.78], [2, 3.2, 38.91], [0.5, 2.8, 54.35], [0.3, 2.4, 59.65], [4.3, 2.9, 60.26], [4.9, 3, 53.65], [5, 2.3, 51.22], [3.6, 2.5, 69.74], [3.8, 2.5, 70.48], [3, 2.3, 63.96], [3.4, 2.1, 56.74], [6.5, 2.6, 65.39], [4.5, 3.7, 59.96], [2.4, 3.3, 61.83], [2.1, 2.8, 55.48], [6, 2.3, 65.39], [7.9, 2.5, 72.7], [1.4, 2.5, 44.22], [0.6, 2.6, 39.57], [3.9, 2.7, 66.04], [7, 1.7, 57.83], [9.1, 2.3, 57.87], [6.9, 3.2, 69.3], [6.4, 2.7, 54.09], [6.4, 2.3, 60.35], [5.5, 1.6, 75.78], [5.6, 1.9, 75.83], [5.8, 2.5, 57.13], [7.5, 2.6, 70.3], [7.8, 4.2, 54.96], [9.8, 2.3, 91.43], [5.6, 2.3, 78.39], [5.8, 1.9, 75.57], [7.6, 1.7, 79.7], [8.6, 1.4, 81.17], [6, 1.3, 76.04], [8.9, 2.8, 88], [8.4, 3.5, 90.96], [7.5, 2.8, 80.22], [10, 2.6, 82.7], [10, 2.8, 85.17], [10, 5.8, 80.39], [10, 4.1, 95.09], [10, 2.3, 99.57], [9.5, 2.9, 83.78], [7, 1.7, 82.26], [8.1, 2.2, 72.91], [10, 1.8, 92.91], [8.3, 2, 82.78], [9, 2.2, 75.3], [8.9, 2.9, 87.78], [7.5, 1.9, 81.22], [9.4, 2, 83.39], [9.5, 2.1, 86.78], [9.9, 2.8, 89.04], [10, 3.3, 91.17], [10, 2.6, 95.96], [9, 1.6, 87.91], [8.9, 1.3, 82.13], [9.6, 2.3, 85.3], [6.8, 2.5, 66.52], [8.3, 1.8, 78.7], [4.5, 2.4, 75.13], [1.3, 2.8, 63.09], [5.1, 1.7, 59.26], [9.9, 2.3, 74.65], [10, 2.6, 95.39], [7.5, 1.8, 83.91], [2.4, 3.1, 62.87], [6.6, 3.4, 62.61], [9.6, 3.1, 87], [10, 3.2, 96.09], [3.9, 3.6, 78.3], [0.1, 1.9, 65.13], [7.9, 2.6, 51.09], [8.9, 2.1, 67.96], [8.1, 1.6, 75.48], [9.6, 2.5, 73.78], [8.8, 4.6, 56.83], [8.4, 3.6, 66.04], [9, 2.9, 77.26], [5.4, 3.3, 63.65], [2.1, 2.3, 56.96], [0.5, 1.8, 56.65], [4, 2, 60.09], [3, 3.1, 67], [3.4, 1.8, 64.13], [6.4, 1.6, 63.39], [6.5, 1.7, 58.83], [0.5, 1.6, 60], [4.6, 1.8, 69.87], [2.5, 1.4, 72.91], [0.3, 1.6, 68.48], [5.4, 1.6, 63.52], [9.9, 2.7, 76.65], [10, 2.7, 98.26], [9.8, 2.9, 84.17], [7.5, 2.1, 73.65], [8.9, 2.1, 76.7], [3.4, 2.5, 70.04], [2.9, 2.5, 65.04], [8.8, 2.1, 83.87], [7.4, 3.8, 74.09], [2.1, 2.2, 68.48], [0, 1.8, 68.61], [2.5, 1.5, 69.17], [7.9, 1.6, 73.22], [5.8, 1.4, 68.83], [8.8, 1.7, 73.78], [1.4, 3.4, 50.39], [0, 2.3, 52.48], [0, 1.8, 55.78], [2.1, 1.8, 63.87], [4.9, 1.7, 66.91], [4.3, 1.9, 75.3], [2.5, 2.7, 66], [0.3, 2.8, 52.17], [0, 2, 55.26], [4.9, 1.9, 58.61], [1.4, 1.7, 69.48], [1.5, 1.7, 70.26], [1.1, 1.8, 64.74], [1.1, 1.9, 65], [4, 2.1, 68.26], [3.4, 1.8, 75.3], [0, 1.7, 56.22], [6.1, 2, 58.35], [0.1, 2.9, 41.65], [0, 1.7, 49.57], [2.6, 1.8, 60.43], [5.9, 1.7, 64], [2.5, 1.5, 67.17], [6.1, 1.5, 76.26], [1.6, 1.3, 74.43], [0.9, 1.8, 62.09], [8.1, 2.2, 77], [8.8, 2.6, 85.26], [5.3, 2.3, 74.96], [6.5, 2, 74.52], [2, 1.7, 71.7], [1.3, 1.6, 64.65], [1, 1.6, 68.35], [3.1, 2.3, 55.87], [1.8, 1.7, 57.65], [3.4, 1.6, 68.26], [8.8, 1.9, 70.26], [3.3, 2.1, 57.09], [3.4, 1.5, 64.3], [5, 3, 65.48], [0.3, 3, 54.39], [2.4, 2.4, 43.48], [8.4, 2.7, 60.87], [8.8, 3.5, 70.48], [1.5, 3.9, 48.13], [1.5, 2.4, 42.48], [1.8, 1.7, 42.74], [6.6, 2.6, 71.3], [6.9, 2.8, 38.83], [2.9, 2.4, 41], [6.1, 2.4, 62.52], [1.8, 2.7, 60.22], [1, 1.4, 56.61], [3.4, 2, 61.57], [6.1, 1.3, 46.35], [0.9, 1.8, 51.74], [7.5, 1.7, 71.39], [6.6, 3.5, 71.35], [1.4, 3.2, 49.09], [7.3, 2.5, 79.09], [4.5, 2, 57.13], [2.3, 2.7, 53.13], [0, 2.6, 46.91], [2.1, 1.7, 55.52], [4.9, 2.2, 55.74], [8.5, 2.7, 82.61], [2.1, 4.4, 61.09], [2, 4.3, 59.87], [0.6, 1.8, 54.74], [0.1, 1.7, 65.09], [1.4, 1.4, 67.74], [0.1, 1.8, 69.09], [3.3, 2.5, 58.3], [3.3, 1.7, 54.22], [1.4, 1.9, 65.48], [7.4, 1.4, 65.7], [4.1, 2, 62.96], [3.4, 2.2, 59.7], [3.1, 2, 72.13], [6.1, 2.1, 70.17], [5.6, 2.1, 83.17], [4.1, 1.7, 73.57], [4.6, 1.7, 67.91], [6.9, 1.8, 71.91], [7.5, 1.7, 73.91], [7.3, 2.1, 75.3], [9.3, 2.8, 82.35], [5.6, 1.7, 69.78], [6.3, 1.8, 74.7], [8.1, 2.3, 84.87], [5.3, 2.3, 76.48], [7.3, 1.8, 79.65], [5.4, 2.2, 74.91], [6.3, 2.3, 73.48], [3.8, 1.9, 75.26], [5, 1.8, 72], [5.3, 2, 67.09], [3.5, 2.4, 63.3], [5, 2.2, 67.87], [4.5, 2.6, 66.7], [9.8, 2.5, 84.39], [8.9, 2.4, 83.04], [9.3, 2.8, 90.22], [4.4, 2.5, 79.48], [5.9, 2.6, 76.91], [7.5, 1.7, 84.7], [8.4, 2.4, 82.35], [9.1, 1.6, 86.09], [4.5, 1.7, 74.22], [7.8, 1.9, 82.87], [9.5, 4.8, 83.22], [4.8, 2.6, 74.65], [4, 2, 76.61], [6.4, 1.9, 76], [4.8, 2.4, 80.26], [5.9, 1.7, 74.65], [8.4, 2.3, 75.22], [5.4, 2, 76.43], [1, 1.7, 71.78], [2.1, 1.6, 65.65], [4.1, 2, 60.87], [7.8, 1.4, 83.65], [6.5, 2, 75.52]]

In [32]:
#feature_set_1_data_2021 = []
#feature_set_1_data_2022 = []

# answersheet = [0] * len(feature_set_1_data_2022[0][0]) + [1] * len(feature_set_1_data_2022[0][1])

answersheet = [0] * 318 + [1] * 47

summeries = summerize_multiple_features(feature_set_1_data_2021)

calculate_f1(answersheet, predict_multiple(feature_set_1_data_2022, summeries))

[(0.003510089788104531, 0.0005974137577093619), (0.0003167881934952407, 3.862492874847255e-05), (0.0, 0.0), (0.0004816614837971132, 7.709936381935178e-05), (0.00030275982783808806, 3.343963899327971e-05), (0.001651222845327119, 0.00015631185294682167)]
[(0.003548968912823833, 0.0005828087670354967), (0.00031680559329087495, 3.862867484059827e-05), (0.0, 0.0), (0.00048179329211319995, 7.710307238902717e-05), (0.0003064641645275767, 3.378962058701019e-05), (0.0017098168631193805, 0.0001525615508955212)]
[(0.003471096950483765, 0.0006002626052279085), (0.00031680559329087495, 3.862867484059827e-05), (0.0, 0.0), (0.00048179329211319995, 7.710307238902717e-05), (0.00030358321205291793, 3.351742628082303e-05), (0.0015536210426176425, 0.0001570822383141205)]
[(0.0035524090757617782, 0.0005746239304291315), (0.00031682273804409434, 3.863239352261345e-05), (0.0, 0.0), (0.00048192420405378803, 7.710656297050022e-05), (0.00030522966004059826, 3.3672978874970493e-05), (0.001701854471266804, 0.0001

(0, 0.0, 0.8712328767123287, 0)

In [33]:
answersheet2 = [0] * 318 + [1] * 47

summeries2 = summerize_multiple_features(feature_set_2_data_2021)

calculate_f1(answersheet2, predict_multiple(feature_set_2_data_2022, summeries2))

[(0.05729948209606415, 0.005750111829217671), (0.1463002998619104, 0.019980092600574706), (0.005140114178649311, 0.000628311144615526)]
[(0.07285371413434424, 0.007339387656766711), (0.1550622777501843, 0.020957421880818113), (0.005385347697770136, 0.0006622005835461813)]
[(0.07360711644965741, 0.007584072995386709), (0.15170527032873712, 0.020663201844707005), (0.005359096845861905, 0.0006580049530915031)]
[(0.04465551638303389, 0.004672444679751933), (0.15484394153322092, 0.02088511682591785), (0.005328060331172172, 0.0006534329653066489)]
[(0.06424128936179262, 0.007508678564218425), (0.150171158467459, 0.020475785711365886), (0.005189226775612917, 0.0006346574601796424)]
[(0.04465551638303389, 0.004672444679751933), (0.1550622777501843, 0.020957421880818113), (0.005197189673562971, 0.0006356961796797196)]
[(0.04465551638303389, 0.004672444679751933), (0.1463002998619104, 0.019980092600574706), (0.005199215083232906, 0.0006359608763409481)]
[(0.07169856047012578, 0.00770325930831989

(0, 0.0, 0.8712328767123287, 0)